# Section 1: Accessing Point of Interest Data with Google Places API

## Learning Goal:
In this section, you’ll learn how to use the Google Places API to fetch points of interest (POIs) in East Harlem. You’ll explore category types, send an API query, filter results spatially, and export your data.

## Step 1: Set Up Your Google Places API Key

**Instructions (only needs to be done once):**
1. Go to the [Google Cloud Console](https://console.cloud.google.com/).
2. Create or select a project.
3. In the navigation menu, go to **APIs & Services > Library**.
4. Search for and enable: **Places API** and **Maps JavaScript API**.
5. Go to **APIs & Services > Credentials** and click **Create Credentials > API key**.
6. Copy the API key (you’ll paste it below).
7. Optional (recommended): Restrict your key to just the Places API and set usage limits.

In [ ]:
# Paste your API key here
GOOGLE_API_KEY = "YOUR_API_KEY_HERE"

## Step 2: Define the Area of Interest

In [ ]:
# Default: East Harlem center and radius
east_harlem_center = (40.8007, -73.9375)
search_radius = 1000  # meters

## 🗂️ Upload a Boundary File for East Harlem

In [ ]:
from google.colab import files
import geopandas as gpd
from shapely.geometry import Point

uploaded = files.upload()
boundary_file = list(uploaded.keys())[0]
east_harlem_poly = gpd.read_file(boundary_file).to_crs(epsg=4326)
east_harlem_poly.explore()

## 🧾 What Place Types Can You Search?

Google uses **Place Types** for consistent queries. 

Examples: `restaurant`, `hospital`, `school`, `grocery_or_supermarket`.

Full list: https://developers.google.com/maps/documentation/places/web-service/supported_types

## 🔎 Step 3: Query for Places

In [ ]:
import requests

def search_places_type(place_type, location, radius, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": api_key,
        "location": f"{location[0]},{location[1]}",
        "radius": radius,
        "type": place_type
    }
    response = requests.get(url, params=params)
    return response.json().get("results", [])

place_type = "school"
results = search_places_type(place_type, east_harlem_center, search_radius, GOOGLE_API_KEY)
print(f"Returned {len(results)} results.")

## 🧠 Understand the Free Tier and Quota Limits

- Free tier: $200/month (~11,000 queries)
- One query returns up to 20 results (can paginate to 60)
- Avoid looping through large batches of places or locations

## 🧹 Filter by Custom Boundary (if uploaded)

In [ ]:
def filter_within_polygon(results, polygon_gdf):
    filtered = []
    for place in results:
        loc = place["geometry"]["location"]
        point = Point(loc["lng"], loc["lat"])
        if polygon_gdf.geometry.contains(point).any():
            place["location_point"] = point
            filtered.append(place)
    return filtered

if 'east_harlem_poly' in globals():
    results = filter_within_polygon(results, east_harlem_poly)
    print(f"Filtered to {len(results)} points inside polygon.")

## 🗺️ Step 4: Visualize on a Map

In [ ]:
!pip install folium
import folium

m = folium.Map(location=east_harlem_center, zoom_start=15, tiles="cartodbpositron")
if 'east_harlem_poly' in globals():
    folium.GeoJson(east_harlem_poly).add_to(m)

for place in results:
    p = place["geometry"]["location"]
    folium.Marker(
        location=[p["lat"], p["lng"]],
        popup=place["name"],
        tooltip=place.get("vicinity", "")
    ).add_to(m)

m

## 💾 Export to GeoJSON

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.DataFrame(results)
df['geometry'] = df.apply(lambda row: Point(row["geometry"]["location"]["lng"], row["geometry"]["location"]["lat"]), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

output_file = "google_places_results.geojson"
gdf.to_file(output_file, driver='GeoJSON')

from google.colab import files
files.download(output_file)